In [2]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:

def evaluar_grad(x, y):
  R = np.sqrt(x**2 + y**2)
  grad_x = -np.cos(R) * (x / R)
  grad_y = -np.cos(R) * (y / R)
  return np.array([grad_x, grad_y])

def gd(theta, epochs, eta):
  for i in range(epochs):
    x, y = theta
    gradient = evaluar_grad(x,y)
    theta -= eta * gradient
  dist = np.linalg.norm(theta)
  return theta, dist

def sgd(theta, data_train, epochs, eta):
  for i in range(epochs):
    np.random.shuffle(data_train)
    for example in data_train:
      x, y = example
      gradient = evaluar_grad(x, y)
      theta = theta - eta * gradient
    dist = np.linalg.norm(theta)
  return theta, dist

def rmsprop(theta, data_train, epochs, eta, decay, epsilon):
  E_g2 = np.zeros_like(theta)
  for i in range(epochs):
    np.random.shuffle(data_train)
    for example in data_train:
      x, y = example
      gradient = evaluar_grad(x, y)
      E_g2 = decay * E_g2 + (1 - decay) * gradient**2
      theta -= eta / (np.sqrt(E_g2) + epsilon) * gradient
    dist = np.linalg.norm(theta)
  return theta, dist

def adam(theta, data_train, epochs, alpha, beta1, beta2, epsilon):
  m = np.zeros_like(theta)
  v = np.zeros_like(theta)
  t = 0

  for epoch in range(epochs):
    np.random.shuffle(data_train)
    for example in data_train:
      x, y = example
      t += 1
      gradient = evaluar_grad(x, y)
      m = beta1 * m + (1 - beta1) * gradient
      v = beta2 * v + (1 - beta2) * (gradient**2)
      m_hat = m / (1 - beta1**t)
      v_hat = v / (1 - beta2**t)
      theta -= alpha * m_hat / (np.sqrt(v_hat) + epsilon)
    dist = np.linalg.norm(theta)
  return theta, dist



In [7]:
np.random.seed(123)
theta_initial = np.array([2.0, 2.0])
x_train_data = np.random.uniform(-6.5, 6.5, 100)
y_train_data = np.random.uniform(-6.5, 6.5, 100)
training_data = list(zip(x_train_data, y_train_data))


theta1, dist_gd = gd(theta_initial, 1000, 0.1)
theta2, dist_sgd = sgd(theta_initial, training_data, 100, 0.01)
theta3, dist_rmsprop = rmsprop(theta_initial, training_data, 100, 0.001, 0.9, 1e-8)
theta4, dist_adam = adam(theta_initial, training_data, 100, 0.001, 0.9, 0.999, 1e-8)

#Dataframe
results_df = pd.DataFrame({
    "Grad. Descent": [dist_gd],
    "Stoch. Grad. Descent": [dist_sgd],
    "RMSProp": [dist_rmsprop],
    "Adam": [dist_adam]
})

# media de las distaciias
average_results = results_df.mean(axis=0)

# Presentación mejorada de la tabla
print("Resultados promedio de cada método:\n")
print(average_results.to_string())

# Determinando el mejor método de optimización
best_method = average_results.idxmin()
print(f"\nEl método de optimización más eficiente es: {best_method}")

Resultados promedio de cada método:

Grad. Descent           1.570796
Stoch. Grad. Descent    1.761105
RMSProp Optimizer       1.358936
Adam Optimizer          1.220721

El método de optimización más eficiente es: Adam Optimizer


In [8]:
iterations = 10000
distances = np.zeros((iterations, 4))
distances_df = pd.DataFrame(distances)
distances_df.columns = ["Grad. Descent", "Stoch. Grad. Descent", "RMSProp Optimizer", "Adam Optimizer"]

theta_initial = np.array([2.0, 2.0])


for i in range(iterations):
    x_data = np.random.uniform(-6.5, 6.5, 100)
    y_data = np.random.uniform(-6.5, 6.5, 100)
    training_data = list(zip(x_data, y_data))
    

    _, distances_df["Grad. Descent"][i] = gd(theta_initial, 1000, 0.1)
    _, distances_df["Stoch. Grad. Descent"][i] = sgd(theta_initial, training_data, 100, 0.01)
    _, distances_df["RMSProp Optimizer"][i] = rmsprop(theta_initial, training_data, 100, 0.001, 0.9, 1e-8)
    _, distances_df["Adam Optimizer"][i] = adam(theta_initial, training_data, 100, 0.001, 0.9, 0.999, 1e-8)

# mejor método para cada iteración
distances_df["Best_Optimizer"] = distances_df.idxmin(axis=1)

# frecuencia y porcentajes
summary_table = distances_df["Best_Optimizer"].value_counts()
percentage = summary_table / iterations
results_table = pd.concat([summary_table, percentage], axis=1)
results_table.columns = ["Frequency", "Percentage"]

print("Resumen de frecuencias de los métodos de optimización:\n")
print(results_table.to_string())

Resumen de frecuencias de los métodos de optimización:

                      Frequency  Percentage
Best_Optimizer                             
Grad. Descent              5027      0.5027
Stoch. Grad. Descent       4663      0.4663
Adam Optimizer              180      0.0180
RMSProp Optimizer           130      0.0130
